# 0. 패키지 불러오기

In [1]:
!pip install catboost
!pip install category_encoders
!pip install optuna
!pip install kmodes
!pip install pycaret

     |████████████████████████████████| 76.1 MB 57.0 MB/s 
     |████████████████████████████████| 82 kB 405 kB/s 
     |████████████████████████████████| 308 kB 4.1 MB/s 
     |████████████████████████████████| 80 kB 9.5 MB/s 
     |████████████████████████████████| 209 kB 54.5 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 149 kB 63.5 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 112 kB 66.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=8fc6651c29acb58266306e001842b4307c938c0025baa8c9e439b13b9ecd5b7f
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
     |████████████████████████████████| 301 kB 4.3 MB/s 
     |████████████████████████████████| 114 kB 49.5 MB/s 
     |████████████████████████████████| 15.5 MB 48.1 MB/s 
     |██████████████

In [2]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm # matplotlib의 font_manager에서 폰트설정을 변경하여 matplotlib에서도 한글 출력을 가능하게 한다.
import seaborn as sns

from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import * # KFold, cross_val_score, train_test_split
from sklearn.metrics import *  # make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import * # LabelEncoder, StandardScaler
from sklearn.svm import * # SVC, SVR
from sklearn.inspection import *
from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import * # PCA

from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from pycaret.classification import *
import random

from lightgbm import *
import lightgbm as lgb
import xgboost as xgb
from datetime import datetime
import calendar
import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. 데이터 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData"

 속성_D_코드.csv	    submission_cat2.csv   test.csv
 속성_H_코드.csv	    submission_cat.csv	 'train2_수정 전.csv'
 속성_L_코드.csv	    submission_lgbm.csv   train2.csv
 참가자_제공_레이아웃.pdf  'test2_수정 전.csv'	  train.csv
 sample_submission.csv	    test2.csv


In [5]:
Rdata_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/train2.csv')
Rdata_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/test2.csv')
Rdata_D = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_D_코드.csv', index_col = 0).T.to_dict()
Rdata_H = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_H_코드.csv', index_col = 0).T.to_dict()
Rdata_L = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_L_코드.csv', index_col = 0).T.to_dict()
print(Rdata_train.shape)
print(Rdata_train.size)
print(Rdata_test.shape)
print(Rdata_test.size)
Rdata_train.head()

(501951, 97)
48689247
(46404, 96)
4454784


,Unnamed: 0,id,person_attribute_a,person_attribute_a_under,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_under,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_세,person_prefer_d_1_소,person_prefer_d_1_중,person_prefer_d_1_대,person_prefer_d_2_세,person_prefer_d_2_소,person_prefer_d_2_중,person_prefer_d_2_대,person_prefer_d_3_세,person_prefer_d_3_소,person_prefer_d_3_중,person_prefer_d_3_대,contents_attribute_d_세,contents_attribute_d_소,contents_attribute_d_중,contents_attribute_d_대,person_prefer_h_1_중,person_prefer_h_1_대,person_prefer_h_2_중,person_prefer_h_2_대,person_prefer_h_3_중,person_prefer_h_3_대,contents_attribute_h_중,contents_attribute_h_대,contents_attribute_l_세,contents_attribute_l_소,contents_attribute_l_중,contents_attribute_l_대,d1_la_match_yn,d1_mi_match_yn,d1_sm_match_yn,d1_sub_match_yn,d1_all_match_yn,d2_la_match_yn,d2_mi_match_yn,d2_sm_match_yn,d2_sub_match_yn,d2_all_match_yn,d3_la_match_yn,d3_mi_match_yn,d3_sm_match_yn,d3_sub_match_yn,d3_all_match_yn,h1_la_match_yn,h1_mi_match_yn,h1_all_match_yn,h2_la_match_yn,h2_mi_match_yn,h2_all_match_yn,h3_la_match_yn,h3_mi_match_yn,h3_all_match_yn,Year,Month,Day,dayofweek,Hour,Minute,Second,dayofyear,season,Hour2,d1_match,d2_match,d3_match,h1_match,h2_match,h3_match
0,0,0,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,2020,1,17,4,12,9,36,17,winter,afternoon,TTTTT,TFFFF,TFFFF,FFF,TFF,FFF
1,1,1,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,True,False,False,2020,6,18,3,17,48,52,170,summer,afternoon,FFFFF,FFFFF,FFFFF,TTF,TFF,TFF
2,2,2,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,2020,7,8,2,20,0,10,190,summer,night,FFFFF,TTFFF,FFFFF,TFF,FFF,TFF
3,3,3,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,2020,1,13,0,18,9,34,13,winter,night,FFFFF,FFFFF,FFFFF,TFF,FFF,FFF
4,4,4,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,2020,3,9,0,20,39,22,69,spring,night,TTTTT,TFFFF,TFFFF,FFF,FFF,FFF


In [13]:
df_train = Rdata_train.copy()
df_test = Rdata_test.copy()
df_D = Rdata_D.copy()
df_H = Rdata_H.copy()
df_L = Rdata_L.copy()

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 97 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Unnamed: 0                  501951 non-null  int64 
 1   id                          501951 non-null  int64 
 2   person_attribute_a          501951 non-null  int64 
 3   person_attribute_a_under    501951 non-null  int64 
 4   person_attribute_b          501951 non-null  int64 
 5   person_prefer_c             501951 non-null  int64 
 6   person_prefer_d_1           501951 non-null  int64 
 7   person_prefer_d_2           501951 non-null  int64 
 8   person_prefer_d_3           501951 non-null  int64 
 9   person_prefer_e             501951 non-null  int64 
 10  person_prefer_f             501951 non-null  int64 
 11  person_prefer_g             501951 non-null  int64 
 12  person_prefer_h_1           501951 non-null  int64 
 13  person_prefer_h_2           5

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 96 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  46404 non-null  int64 
 1   id                          46404 non-null  int64 
 2   person_attribute_a          46404 non-null  int64 
 3   person_attribute_a_under    46404 non-null  int64 
 4   person_attribute_b          46404 non-null  int64 
 5   person_prefer_c             46404 non-null  int64 
 6   person_prefer_d_1           46404 non-null  int64 
 7   person_prefer_d_2           46404 non-null  int64 
 8   person_prefer_d_3           46404 non-null  int64 
 9   person_prefer_e             46404 non-null  int64 
 10  person_prefer_f             46404 non-null  int64 
 11  person_prefer_g             46404 non-null  int64 
 12  person_prefer_h_1           46404 non-null  int64 
 13  person_prefer_h_2           46404 non-null  in

# 2. 범주형 encoding

In [14]:
df_train.dtypes.value_counts()

int64     65
bool      24
object     8
dtype: int64

int형에 속하는 컬럼들에 특징 값 분포
*   특징 값 분포를 통해서 Label Encoding과 Ont-Hot Encoding 방식을 결정
*   가장 많은 특징 값을 갖는 것이 'person_rn' -> 300177


*   동일한 특징 값을 갖는 특징들은 ['person_prefer_f', 'person_prefer_g', 'Y'] 라는 것을 파악 ->  이 변수들은 볼 필요가 없다.
  *   모든 값에 대해서 동일한 값을 갖는 것을 주목해야 한다.
  *   이러한 특징 값들은 추후 모델 학습에서 Drop 시키고 학습을 실시 하겠습니다.

Ont-Hot Encoding
1. 순서가 없을 때 (ex. 국가명)
2. 고유값의 개수가 많지 않으면 효율적

Label Encoding
1. 순서의 의미가 있을때 (유치원, 초등학교, 대학교 같은 등급, 사원, 대리, 과장, 부장 같은 직급?? )
2. 고유값의 개수가 많을때 (One-hot Encoding은 당연히 메모리 소비가 많으므로 효율적이진 못하다. )
3. 선형 회귀와 같은 ML알고리즘에는 적용 불가
4. 트리 계열의 ML 알고리즘은 숫자의 이러한 특성을 반영하지 않으므로 가능
5. LabelEncoder는 dataframe의 열마다 접근해야 한다.

In [15]:
df_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)[df_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0) == 1]

person_prefer_f    1
person_prefer_g    1
Year               1
dtype: int64

In [11]:
df_train.columns

Index(['Unnamed: 0', 'id', 'person_attribute_a', 'person_attribute_a_under',
       'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1',
       'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e',
       'person_prefer_f', 'person_prefer_g', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세'

In [12]:
# for column in df_train.select_dtypes(include = 'bool').columns :
#     df_train[column] = df_train[column].astype(int)
    
# for column in df_test.select_dtypes(include = 'bool').columns :
#     df_test[column] = df_test[column].astype(int)

In [16]:
df_train2 = df_train.drop(['person_prefer_f', 'person_prefer_g', 'Year', 'Unnamed: 0', 'id'], axis = 1)
df_test2 = df_test.drop(['person_prefer_f', 'person_prefer_g', 'Year', 'Unnamed: 0', 'id'], axis = 1)
print(df_train2.shape)
print(df_test2.shape)

(501951, 92)
(46404, 91)


In [18]:
df_train2.dtypes.value_counts()

int64     60
bool      24
object     8
dtype: int64

In [19]:
df_train2.select_dtypes(include = 'object').columns

Index(['season', 'Hour2', 'd1_match', 'd2_match', 'd3_match', 'h1_match',
       'h2_match', 'h3_match'],
      dtype='object')

In [20]:
df_train2.columns

Index(['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b',
       'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2',
       'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세',
       'contents_attribute_d_소', 'contents_attribute_d_중',
    

In [21]:
from pycaret.classification import *

clf = setup(df_train2, target = 'target', train_size = 0.85)

,Description,Value
0,session_id,8079
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(501951, 92)"
5,Missing Values,False
6,Numeric Features,36
7,Categorical Features,55
8,Ordinal Features,False
9,High Cardinality Features,False


In [23]:
best5 = compare_models(fold = 5, sort = 'F1', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.5066,0.5071,0.9235,0.5034,0.6512,0.0142,0.0238,15.424
catboost,CatBoost Classifier,0.6246,0.6740,0.6627,0.6153,0.6381,0.2493,0.2500,63.544
lightgbm,Light Gradient Boosting Machine,0.6164,0.6637,0.6640,0.6058,0.6335,0.2328,0.2339,8.256
rf,Random Forest Classifier,0.6100,0.6555,0.6247,0.6063,0.6154,0.2200,0.2201,82.984
gbc,Gradient Boosting Classifier,0.6025,0.6463,0.6324,0.5962,0.6138,0.2051,0.2055,174.632
et,Extra Trees Classifier,0.6097,0.6540,0.6196,0.6070,0.6132,0.2194,0.2195,113.662
ridge,Ridge Classifier,0.5991,0.0000,0.6225,0.5942,0.6080,0.1983,0.1985,1.372
lda,Linear Discriminant Analysis,0.5991,0.6391,0.6226,0.5942,0.6080,0.1983,0.1985,20.990
ada,Ada Boost Classifier,0.5954,0.6343,0.6063,0.5928,0.5995,0.1908,0.1909,38.216
dt,Decision Tree Classifier,0.5441,0.5441,0.5429,0.5437,0.5433,0.0882,0.0882,13.834


In [24]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'F1')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5841,0.6483,0.8029,0.5581,0.6585,0.1686,0.1875
1,0.5823,0.6616,0.8924,0.5505,0.6810,0.1653,0.2106
2,0.5825,0.6560,0.8612,0.5526,0.6732,0.1655,0.1993
3,0.5816,0.6642,0.9073,0.5491,0.6842,0.1638,0.2158
4,0.5820,0.6567,0.8639,0.5521,0.6737,0.1646,0.1992
Mean,0.5825,0.6573,0.8656,0.5525,0.6741,0.1655,0.2025
SD,0.0008,0.0054,0.0358,0.0031,0.0089,0.0016,0.0099


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.585,0.6655,0.8944,0.5533,0.6837,0.1685,0.2146


In [25]:
custom_model = compare_models(fold = 5, sort = 'F1', n_select = 5, 
                              include = ['catboost', 'lightgbm', 'rf', 'gbc', 'et'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.6246,0.6740,0.6627,0.6153,0.6381,0.2493,0.2500,63.632
lightgbm,Light Gradient Boosting Machine,0.6164,0.6637,0.6640,0.6058,0.6335,0.2328,0.2339,8.786
rf,Random Forest Classifier,0.6100,0.6555,0.6247,0.6063,0.6154,0.2200,0.2201,85.796
gbc,Gradient Boosting Classifier,0.6025,0.6463,0.6324,0.5962,0.6138,0.2051,0.2055,171.846
et,Extra Trees Classifier,0.6097,0.6540,0.6196,0.6070,0.6132,0.2194,0.2195,114.906


In [ ]:
blended_custom = blend_models(estimator_list = custom_model, fold = 5, optimize = 'F1')
pred_holdout_custom = predict_model(blended_custom)

IntProgress(value=0, description='Processing: ', max=6)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,07:52:07
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Voting Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 
final_model_custom = finalize_model(blended_custom)

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model_custom])
prections = prep_pipe.predict_proba(df_test2)
prections

In [ ]:
scores = []
for threshold in range(1, 20) :
    threshold = threshold / 20
    pred = prep_pipe.predict_proba(X_train)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)

print(scores)
print(np.mean(scores))

# 0.7512865208988593

In [ ]:
# 제출

submission_preds = prections
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/sample_submission.csv')
submission['target'] = submission_preds
submission.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/submission_pycaret.csv', index = False)